# Library Migration in Python
1. 筛选出100个star以上的带有requirements.txt的project
2. 根据git tag获取tag信息以及SHA
3. git checkout SHA，cat requirements.txt获取依赖信息
4. 每个tag与最近的版本进行比较，提取依赖信息变化
5. 构造迁移规则，计算四个指标，提取出最可能的规则
6. 根据规则分析迁移频率以及特点
7. 实现库迁移推荐算法

In [66]:
from openpyxl import Workbook, load_workbook

# need fix
def get_add_rem():
    ws = load_workbook('data/migration_changes_without_verchanges.xlsx', read_only=True)['Sheet1']
    commit = ws.cell(2, 2).value
    now_commit = ''
    add_rem = []
    add = []
    rem = []
    for s in ws.rows:
        print(s[0].value)
        break
        now_commit = s[1].value
        if now_commit != commit:
            if add and rem:
                add_rem.append({'add':add, 'rem': rem})
            add = []
            rem = []
            commit = now_commit
        else:
            if s[3].value == 'add':
                add.append(s[4].value)
            elif s[3].value == 'rem':
                rem.append(s[6].value)
    for m in add_rem[:50]:
        print('add: {}'.format(m['add']))
        print('rem: {}'.format(m['rem']))



  repoName   version                   publishedTime
0  ansible     2.9.2  Wed Dec 04 23:12:31 2019 +0000
1  ansible     2.9.1  Thu Nov 14 01:10:36 2019 +0000
2  ansible     2.8.7  Thu Nov 14 00:30:26 2019 +0000
3  ansible    2.7.15  Wed Nov 13 21:41:46 2019 +0000
4  ansible     2.9.0  Thu Oct 31 16:59:26 2019 +0000
5  ansible  2.9.0rc5  Wed Oct 23 19:53:05 2019 +0000
6  ansible  2.9.0rc4  Thu Oct 17 18:49:20 2019 +0000
7  ansible     2.8.6  Thu Oct 17 18:10:34 2019 +0000
8  ansible    2.7.14  Thu Oct 17 17:55:06 2019 +0000
9  ansible    2.6.20  Thu Oct 17 17:11:24 2019 +0000


,repoName,version,publishedTime
0,ansible,0.0.1,2012-03-13 23:27:13+00:00
1,ansible,0.0.2,2012-04-04 00:20:55+00:00
2,ansible,0.1.0,2012-03-08 19:12:58+00:00
3,ansible,0.3.0,2012-04-24 01:13:06+00:00
4,ansible,0.3.1,2012-05-01 23:03:55+00:00
...,...,...,...
185,ansible,2.9.17,2021-01-19 00:19:16+00:00
186,ansible,2.9.18,2021-02-19 00:33:27+00:00
187,ansible,2.9.19,2021-03-16 00:54:13+00:00
188,ansible,2.9.2,2019-12-04 23:18:50+00:00


In [47]:
import pandas as pd
df = pd.read_excel('data/project_version_with_commit.xlsx')
df[df['repoName']=='ansible']


,repoName,version,commit,date
80926,ansible,0.1.0,56de2e112a97f312c7a07c89e1ce5de74a2637f8,2012-03-08 19:12:58+00:00
80927,ansible,0.0.1,2e1b59a9d2440be500cbb73117ffcf60a5bc137b,2012-03-13 23:27:13+00:00
80928,ansible,0.0.2,09e690fd7c303466179cdf417f50de64875a9657,2012-04-04 00:20:55+00:00
80929,ansible,0.3.0,4d6251099756c98a4b7224571d4f7d2128c8f4d2,2012-04-24 01:13:06+00:00
80930,ansible,0.3.1,5a156d6817e1f4fc77801bad22e4ddbd0719b57e,2012-05-01 23:03:55+00:00
...,...,...,...,...
81117,ansible,2.10.6,9ea56ff2925a2d6afe62333340570b44e59e50a1,2021-02-19 00:31:43+00:00
81118,ansible,2.9.18,4455e60166a8271771d96c24f47b6714130d200f,2021-02-19 00:33:27+00:00
81119,ansible,2.8.19,1205ad6a1a108376654d128ac903049168bf3d35,2021-02-19 00:33:42+00:00
81120,ansible,2.9.19,98cb6fe4ebbc8f04cb72f5e2d86c91baab8d9836,2021-03-16 00:54:13+00:00


In [80]:
# 获取每个版本的requirements.txt
import pandas as pd
import subprocess
import pkg_resources
from dependency import Dependency
from get_dependency_change import Diff
import numpy as np
from parallel import parallel
import warnings
warnings.filterwarnings("ignore")
# 并行
# df = pd.read_excel('data/project_version_with_commit.xlsx')
# 过去的比当前版本小的最大版本
# 获取版本间依赖变化
def get_requirements_change(s):
    repo = s[0]
    date = s[3]
    commit = s[2]
    # s行对应的仓库的所有过去版本
    df_repo = df[(df['repoName']==repo) & (df['date'] < date)].values
    version_num = len(df_repo)
    
    # 过去的比当前版本小的最大版本
    now_version = pkg_resources.parse_version(s[1])
    last_version = pkg_resources.parse_version('0.0.0')
    last_commit = s[2]
    for i in range(0, version_num):
        temp_version = pkg_resources.parse_version(df_repo[i][1])
        if temp_version < now_version and temp_version > last_version:
            last_version = temp_version
            last_commit = df_repo[i][2]
#     print(now_version, last_version, last_commit)
    
    # git diff tag1 tag2 requirements.txt
    df_dc = pd.DataFrame(columns = ['repoName', 'commit', 'date', 'type', 'l1', 'v1', 'l2', 'v2'])
    cmd = 'cd repos/{} && git diff {} {} requirements.txt  '.format(repo, last_version, now_version)
    p = subprocess.Popen(cmd,shell=True,stdout=subprocess.PIPE)  
    out,err = p.communicate() 
    r = Diff(repo, out.decode().split('\n'))
    r.analyse()
    common_l = set([x.project_name for x in r.adds]) & set([x.project_name for x in r.rems])
    for l in common_l:
        data = {
            'repoName': repo,
            'commit': commit,
            'date': date,
            'type': 'verchange', 
            'l1': l, 
            'v1': [';'.join(['' + spec[0] + spec[1] for spec in x._specs]) for x in r.adds if x.project_name == l][0],
            'l2': l, 
            'v2': [';'.join(['' + spec[0] + spec[1] for spec in x._specs]) for x in r.rems if x.project_name == l][0]
        }
        df_dc = df_dc.append(data, ignore_index=True)
    for add in r.adds:
        if add.project_name in common_l or add.project_name is None:
            continue
        data = {
            'repoName': repo,
            'commit': commit,
            'date': date,
            'type': 'add', 
            'l1': add.project_name, 
            'v1': ';'.join(['' + spec[0] + spec[1] for spec in add._specs]),
            'l2': np.nan, 
            'v2': np.nan
        }
        df_dc = df_dc.append(data, ignore_index=True)
    for rem in r.rems:
        if rem.project_name in common_l or rem.project_name is None:
            continue
        data = {
            'repoName': repo,
            'commit': commit,
            'date': date,
            'type': 'rem', 
            'l1': np.nan, 
            'v1': np.nan,
            'l2': rem.project_name, 
            'v2': ';'.join(['' + spec[0] + spec[1] for spec in rem._specs])
        }
        df_dc = df_dc.append(data, ignore_index=True)
    return df_dc

df_tdc = parallel(get_requirements_change, 96, df.values)
df_tdc.to_excel('data/new_migration_changes.xlsx', index=False)
df_tdc.head(10)
    


计算进度: 100%|██████████| 82014/82014 [16:53<00:00, 80.95it/s, 并行函数=get_requirements_change, 计算花销=1013s]


,repoName,commit,date,type,l1,v1,l2,v2
0,ncbi-genome-download,a078d93517236acfacb0ec1a633e6728384390b5,2016-09-09 17:11:51+00:00,add,pyOpenSSL,>=16.0.0,NaN,NaN
1,ncbi-genome-download,a078d93517236acfacb0ec1a633e6728384390b5,2016-09-09 17:11:51+00:00,add,ndg-httpsclient,>=0.4.0,NaN,NaN
2,ncbi-genome-download,26de45d3b18059269b77b5a342f27e7e53c1d74a,2016-11-30 15:45:49+00:00,rem,NaN,NaN,requests,
3,ncbi-genome-download,26de45d3b18059269b77b5a342f27e7e53c1d74a,2016-11-30 15:45:49+00:00,rem,NaN,NaN,pyOpenSSL,>=16.0.0
4,ncbi-genome-download,26de45d3b18059269b77b5a342f27e7e53c1d74a,2016-11-30 15:45:49+00:00,rem,NaN,NaN,ndg-httpsclient,>=0.4.0
5,pylint-odoo,6a8f6960ea527ab2e33ccaff2e0b38aa34e85a19,2016-05-16 16:32:04+00:00,add,astroid,==1.3.8,NaN,NaN
6,pylint-odoo,34d5e0eab0694058fe15280d11210c3b9c7f3cd6,2016-09-27 15:41:12+00:00,add,isort,==4.2.5,NaN,NaN
7,pylint-odoo,57fa5a766f593f5be14f56314fb3b971b8a774ad,2016-11-26 13:18:58+00:00,verchange,pylint,==1.6.4,pylint,==1.4.4
8,pylint-odoo,57fa5a766f593f5be14f56314fb3b971b8a774ad,2016-11-26 13:18:58+00:00,verchange,astroid,==1.4.8,astroid,==1.3.8
9,pylint-odoo,57fa5a766f593f5be14f56314fb3b971b8a774ad,2016-11-26 13:18:58+00:00,verchange,pylint-plugin-utils,==0.2.4,pylint-plugin-utils,==0.2.3


In [81]:
import pandas as pd
pd.read_excel('data/new_migration_changes.xlsx').head(10)

,repoName,commit,date,type,l1,v1,l2,v2
0,ncbi-genome-download,a078d93517236acfacb0ec1a633e6728384390b5,2016-09-09 17:11:51+00:00,add,pyOpenSSL,>=16.0.0,NaN,NaN
1,ncbi-genome-download,a078d93517236acfacb0ec1a633e6728384390b5,2016-09-09 17:11:51+00:00,add,ndg-httpsclient,>=0.4.0,NaN,NaN
2,ncbi-genome-download,26de45d3b18059269b77b5a342f27e7e53c1d74a,2016-11-30 15:45:49+00:00,rem,NaN,NaN,requests,NaN
3,ncbi-genome-download,26de45d3b18059269b77b5a342f27e7e53c1d74a,2016-11-30 15:45:49+00:00,rem,NaN,NaN,pyOpenSSL,>=16.0.0
4,ncbi-genome-download,26de45d3b18059269b77b5a342f27e7e53c1d74a,2016-11-30 15:45:49+00:00,rem,NaN,NaN,ndg-httpsclient,>=0.4.0
5,pylint-odoo,6a8f6960ea527ab2e33ccaff2e0b38aa34e85a19,2016-05-16 16:32:04+00:00,add,astroid,NaN,NaN,NaN
6,pylint-odoo,34d5e0eab0694058fe15280d11210c3b9c7f3cd6,2016-09-27 15:41:12+00:00,add,isort,NaN,NaN,NaN
7,pylint-odoo,57fa5a766f593f5be14f56314fb3b971b8a774ad,2016-11-26 13:18:58+00:00,verchange,pylint,NaN,pylint,NaN
8,pylint-odoo,57fa5a766f593f5be14f56314fb3b971b8a774ad,2016-11-26 13:18:58+00:00,verchange,astroid,NaN,astroid,NaN
9,pylint-odoo,57fa5a766f593f5be14f56314fb3b971b8a774ad,2016-11-26 13:18:58+00:00,verchange,pylint-plugin-utils,NaN,pylint-plugin-utils,NaN
